<a href="https://colab.research.google.com/github/RogerHeederer/ForAllPytorch/blob/main/Pad_Pack_Sequence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RNN과 LSTM 계열 모델에서 sequence batch를 활용하는 팁에 대해서 설명

In [34]:
import torch
import numpy as np
from torch.nn.utils.rnn import pad_sequence, pack_sequence, pack_padded_sequence, pad_packed_sequence

In [35]:
data = ['hello world', 'midnight', 'calculation', 'path', 'short circuit']

#단어 집합 만들기
char_set = []

for seq in data:
    for char in seq:
        char_set += char

char_set = ['<pad>'] + list(set(char_set))

#위에 구현을 한 줄로도 표현 가능하다.
#char_set = [<'pad'>] + list(set(char for seq in data for char in seq))

In [36]:
char2idx = {char: idx for idx, char in enumerate(char_set)}

In [37]:
print('char_set:', char_set)
print('char2idx:', char2idx)
print('char_set length:', len(char_set))

char_set: ['<pad>', ' ', 'i', 'a', 't', 'e', 'r', 'm', 'p', 's', 'h', 'g', 'w', 'l', 'd', 'o', 'n', 'c', 'u']
char2idx: {'<pad>': 0, ' ': 1, 'i': 2, 'a': 3, 't': 4, 'e': 5, 'r': 6, 'm': 7, 'p': 8, 's': 9, 'h': 10, 'g': 11, 'w': 12, 'l': 13, 'd': 14, 'o': 15, 'n': 16, 'c': 17, 'u': 18}
char_set length: 19


In [63]:
X = []
for seq in data:
    X.append(torch.LongTensor([char2idx[char] for char in seq]))

#위 코드를 한줄로 구현
#X = [torch.LongTensor([char2idx[char] for char in seq]) for seq in data]

In [64]:
for sequence in X:
    print(sequence)

tensor([10,  5, 13, 13, 15,  1, 12, 15,  6, 13, 14])
tensor([ 7,  2, 14, 16,  2, 11, 10,  4])
tensor([17,  3, 13, 17, 18, 13,  3,  4,  2, 15, 16])
tensor([ 8,  3,  4, 10])
tensor([ 9, 10, 15,  6,  4,  1, 17,  2,  6, 17, 18,  2,  4])


In [65]:
# 이렇게 길이가 제각각이다
lengths = [len(seq) for seq in X]
print('lengths:', length)

lengths: [11, 8, 11, 4, 13]


## pad_sequence 함수를 사용하여 PaddedSequence(텐서) 만들기
- 구성된 텐서 리스트의 Max값을 알아서 캐치하여, 부족한 리스트들은 뒤에 0을 패딩시켜줌


In [66]:
padded_sequence = pad_sequence(X, batch_first=True) # X는 반드시 tensor의 리스트여야 함. 그냥 Tensor가 아님
print(padded_sequence)
print(padded_sequence.shape) # 배치 5가 가장 앞에 위치함

tensor([[10,  5, 13, 13, 15,  1, 12, 15,  6, 13, 14,  0,  0],
        [ 7,  2, 14, 16,  2, 11, 10,  4,  0,  0,  0,  0,  0],
        [17,  3, 13, 17, 18, 13,  3,  4,  2, 15, 16,  0,  0],
        [ 8,  3,  4, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 9, 10, 15,  6,  4,  1, 17,  2,  6, 17, 18,  2,  4]])
torch.Size([5, 13])


##PackedSequence

패딩 토큰을 추가해서 최대 길이에 맞게 텐서 리스트를 만드는게 아니고,

패딩을 추가하지 않고 정확히 주어진 시퀀스 길이까지만 모델이 연산 하게끔 만드는 파이토치 자료구조이다. 즉 pad 토큰을 연산하는 자원 낭비를 막을 수 있다.

다만 조건이 있는데, 주어지는 list of Tensor는 길이에 따른 내림차순 정렬로 되어 있어야 함

In [71]:
# 0~4까지 범주의 숫자를 정렬하는데, 내림 차순 정렬을 할꺼고, 기준은 lengths.__getitem__이다
# lengths.__getitem__은 lengths 리스트가 가지고 있는 깂들의 인덱스를 돌려준다.
sorted_idx = sorted(range(5), key=lengths.__getitem__, reverse=True)

In [72]:
lengths
# 0  1  2  3  4
#11  8 11  4  13

# 내림차순 정렬하면 
#    13  11  11  8  4
#idx 4   0   2   1  3

[11, 8, 11, 4, 13]

In [73]:
# X[0] X[1] X[2] X[3] X[4]들 중에서 길이 내림차순으로 정렬 후 담아놓은 idx 리스트
sorted_idx

[4, 0, 2, 1, 3]

In [77]:
sorted_X = [X[idx] for idx in sorted_idx]

In [78]:
sorted_X

[tensor([ 9, 10, 15,  6,  4,  1, 17,  2,  6, 17, 18,  2,  4]),
 tensor([10,  5, 13, 13, 15,  1, 12, 15,  6, 13, 14]),
 tensor([17,  3, 13, 17, 18, 13,  3,  4,  2, 15, 16]),
 tensor([ 7,  2, 14, 16,  2, 11, 10,  4]),
 tensor([ 8,  3,  4, 10])]

In [124]:
packed_sequence = pack_sequence(sorted_X)
print(packed_sequence) # 위 2차원 텐서를 column 기준으로 세로로 읽으면 된다.

PackedSequence(data=tensor([ 9, 10, 17,  7,  8, 10,  5,  3,  2,  3, 15, 13, 13, 14,  4,  6, 13, 17,
        16, 10,  4, 15, 18,  2,  1,  1, 13, 11, 17, 12,  3, 10,  2, 15,  4,  4,
         6,  6,  2, 17, 13, 15, 18, 14, 16,  2,  4]), batch_sizes=tensor([5, 5, 5, 5, 4, 4, 4, 4, 3, 3, 3, 1, 1]), sorted_indices=None, unsorted_indices=None)


In [93]:
#단위 행렬 만드는 함수
eye = torch.eye(5)
eye

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]])

In [102]:
eye[[2,3,4]] #eye[2], eye[3], eye[4]를 뽑아온건데, 2,3,4의 원핫 인코딩 표현으로도 볼 수 있다

tensor([[0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]])

In [104]:
#그럼 가지고 있는 char_set 길이만큼 공간을 단위행렬로 확보 한 다음에 원핫 인코딩을 만들어보자
eye = torch.eye(len(char_set))

In [105]:
padded_sequence

tensor([[10,  5, 13, 13, 15,  1, 12, 15,  6, 13, 14,  0,  0],
        [ 7,  2, 14, 16,  2, 11, 10,  4,  0,  0,  0,  0,  0],
        [17,  3, 13, 17, 18, 13,  3,  4,  2, 15, 16,  0,  0],
        [ 8,  3,  4, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 9, 10, 15,  6,  4,  1, 17,  2,  6, 17, 18,  2,  4]])

In [106]:
embedded_tensor = eye[padded_sequence]

In [109]:
embedded_tensor[0] # 10, 5, 13, 13, 15 , 1, 12, 15, 6, 13, 14, 0, 0

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [111]:
embedded_tensor.shape #(batch_size, max_sequence, number_of_input_tokens)

torch.Size([5, 13, 19])

In [112]:
sorted_idx

[4, 0, 2, 1, 3]

In [115]:
# one-hot embedding using PackedSequence
embedded_packed_seq = pack_sequence([eye[X[idx]] for idx in sorted_idx])
embedded_packed_seq.data.shape

torch.Size([47, 19])

In [116]:
rnn = torch.nn.RNN(input_size=len(char_set), hidden_size=30, batch_first=True)

Padded Sequence 데이터 넣어보기

In [126]:
rnn_output, hidden = rnn(embedded_tensor)
print(rnn_output.shape)
print(hidden.shape)

torch.Size([5, 13, 30])
torch.Size([1, 5, 30])


Packed Sequence 데이터 넣어보기

In [127]:
rnn_output, hidden = rnn(embedded_packed_seq)
print(rnn_output.data.shape)
print(hidden.data.shape)

torch.Size([47, 30])
torch.Size([1, 5, 30])
